#### Add dependencies

In [1]:
%AddDeps org.apache.spark spark-mllib_2.10 1.6.2
%AddDeps com.github.haifengl smile-core 1.1.0 --transitive
%AddDeps io.reactivex rxscala_2.10 0.26.1 --transitive
%AddDeps com.chuusai shapeless_2.10 2.3.0 --repository https://oss.sonatype.org/content/repositories/releases/
%AddDeps org.tmoerman plongeur-spark_2.10 0.3.8 --repository file:/Users/tmo/.m2/repository

Marking org.apache.spark:spark-mllib_2.10:1.6.2 for download
Preparing to fetch from:
-> file:/var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps830962751445828928/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps830962751445828928/https/repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.10/1.6.2/spark-mllib_2.10-1.6.2.jar
Marking com.github.haifengl:smile-core:1.1.0 for download
Preparing to fetch from:
-> file:/var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps830962751445828928/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps830962751445828928/https/repo1.maven.org/maven2/com/github/haifengl/smile-graph/1.1.0/smile-graph-1.1.0.jar
-> New file at /var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps830962751445828928/https/repo1.maven.org/maven2/com/github/haifengl/smile-math/1.1.0/smile-math-1.1.0.jar
-> New file at /va

In [2]:
%addjar http://localhost:8888/nbextensions/declarativewidgets/urth-widgets.jar

Starting download from http://localhost:8888/nbextensions/declarativewidgets/urth-widgets.jar
Finished download of urth-widgets.jar


In [3]:
System.getProperty("java.version")

1.8.0_74

#### Import classes

In [4]:
import rx.lang.scala.{Observer, Subscription, Observable}
import rx.lang.scala.subjects.PublishSubject
import rx.lang.scala.subjects._

import shapeless.HNil

import org.tmoerman.plongeur.tda._
import org.tmoerman.plongeur.tda.Model._
import org.tmoerman.plongeur.tda.cluster.Clustering._
import org.tmoerman.plongeur.tda.cluster.Scale._

import declarativewidgets._
initWidgets

import declarativewidgets.WidgetChannels.channel

#### Import polymer elements

In [5]:
%%html
<link rel='import' href='urth_components/paper-slider/paper-slider.html' 
        is='urth-core-import' package='PolymerElements/paper-slider'>

#### TDA Machine

Instantiate a `PublishSubject`. This stream of `TDAParams` instances represents the input of a `TDAMachine`. The `PublishSubject` listens to changes and sets these to the channel `"ch_TDA_1"` under the `"params"` key.

*TODO: unsubscribe previous on re-evaluation*

In [6]:
val in = PublishSubject[TDAParams]

val in_sub = in.subscribe(p => channel("ch_TDA_1").set("params", p.toString))

Create an initial `TDAParams` instance. In the same cell, we submit the instance to the `PublishSubject`.

In [15]:
val tdaParams =
      TDAParams(
        lens = TDALens(
          Filter("feature" :: 0 :: HNil, 10, 0.5),
          Filter("feature" :: 1 :: HNil, 13, 0.5)),
        clusteringParams = ClusteringParams(),
        scaleSelection = histogram(10))

in.onNext(tdaParams)

Cluster(bedcde4d-ef48-48f2-b855-875d7a918be0)
Cluster(fa5314d0-3b26-4939-ba01-756ee3c2a902)
Cluster(cf186e57-a0d9-43f3-b1e6-8a0b84e8aab1)
Cluster(15f92602-e048-4212-8209-c4ecd3e7d6a4)
Cluster(cb1f8a80-1125-478f-ab6d-22fe0da2ae95)
Cluster(95d01a35-1859-42f7-8b39-1c80833a783b)
Cluster(6a5bf31f-7028-4a8b-b5bc-7ad12e60b3a7)
Cluster(2d480938-b352-435a-9069-f118794eef00)
Cluster(c91bcec2-825c-4024-a38e-8e66c605e7ae)
Cluster(7dc01667-17c7-4edc-ac77-3e3b08e2b90d)
Cluster(9a49209f-e856-45a9-91ff-f3e001bc054d)
Cluster(d0414fdd-9d45-4a0f-9131-5aeb1275dda4)
Cluster(d110e326-9beb-4d90-b55b-6d2a7a2c030e)
Cluster(5b2016cd-d9d2-4e42-ac12-69cbf36673da)
Cluster(bf988778-2697-478c-862c-9be9a4112657)
Cluster(d29ee946-fa42-4d23-a75b-3b0f36b7be74)
Cluster(36801a34-0f85-4ab3-8bef-1055281d0e90)
Cluster(6f3453ce-64ff-4dc2-9067-44ba93d8cf32)
Cluster(8b03281f-8025-4dad-8d18-419c98bde438)
Cluster(a4f30b04-c038-4374-9c41-16ba43333410)
Cluster(dfaec01f-475f-4cda-98a9-4c8d63842fdc)
Cluster(6c032e28-43ea-4916-84f4-e9

Specify alternative parameters.

In [16]:
val tdaParams_ALT =
      TDAParams(
        lens = TDALens(
          Filter("feature" :: 0 :: HNil, 10, 0.5)),
        clusteringParams = ClusteringParams(),
        scaleSelection = histogram(10))

in.onNext(tdaParams_ALT)

Cluster(0bffe3ee-306d-4f8f-b2b3-03c1e62ad7cb)
Cluster(24f4ddfd-4113-4de0-b97a-156210e0fe5e)
Cluster(113062af-8d82-4e26-8bc0-99956e0763db)
Cluster(1ac6001d-36c5-4d54-895f-60555ae43b80)
Cluster(5ea6515e-fd96-4d20-9e25-fcc97f138927)
Cluster(127030c8-6666-4ef0-9348-2d52f1e02edc)
Cluster(d4cd7cbf-1604-45fb-95df-a0c7a60ac069)
Cluster(a8cddab7-f7ee-42c2-b7c7-85a5efedca6f)
Cluster(090aa2a2-2277-40bf-a0c6-fa13eeecef28)
Cluster(b5254a6e-4a8f-407e-af3f-03c388fe70cf)
Cluster(be8cb3e6-32e7-41ea-b362-e2e1db0738b0)
Cluster(8b442f11-567f-48a4-892a-633e73fa1e3c)
Cluster(b226fdc5-5a37-40e9-9da6-4abc237d1243)
Cluster(80a71f2a-aa3b-4f25-acdb-0d71f6b6190a)
Cluster(cca16ae4-ee30-44ad-a008-9a399efa5999)
Cluster(8c09f0c9-f6b8-40d8-9826-7ab76209140e)
Cluster(7bc8c624-4b29-4a1b-a5b7-0fc14e6e0301)


For the sake of illustration, we create an html snippet that listens to changes on the `"ch_TDA_1"` channel and displays the value of the `"params"` key.

In [9]:
%%html
<template is='urth-core-bind' channel='ch_TDA_1'>  
    <div style='background: #FFB; padding: 10px;'>
        <span style='font-family: "Courier"'>[[params]]</span>
    </div>
</template>

[[params]]

Notice that when we evaluate the `TDAParams` instantiation cells, the output of the yellow box changes.

#### Inititalize rdd

In [10]:
import org.apache.spark.rdd.RDD
import org.apache.commons.lang.StringUtils.trim
import org.apache.spark.mllib.linalg.Vectors.dense

def readCircle(file: String) = 
    sc.
        textFile(file).
        map(_.split(",").map(trim)).
        zipWithIndex.
        map{ case (Array(x, y), idx) => dp(idx, dense(x.toDouble, y.toDouble))}         

In [11]:
val data_path = "/Users/tmo/Work/batiskav/projects/plongeur/scala/plongeur-spark/src/test/resources/data/"

val circle_1k_path = data_path + "circle.1k.csv"

val rdd = readCircle(circle_1k_path)

In [12]:
val ctx = TDAContext(sc, rdd)

In [13]:
val machine_out: Observable[(TDAParams, TDAResult)] = TDAMachine.run(ctx, in)

In [14]:
val out_sub = machine_out.subscribe(
    onNext = (t) => t match {case (p, r) => println(r.clusters.mkString("\n"))},
    onError = (e) => println("Error in TDA machine: ", e))

In [33]:
out_sub.unsubscribe()